# Parameter Sweeps and Batch Processing

Power system studies frequently require running many simulations with systematically varied parameters. You might want to understand how load levels affect voltage stability, how different fault locations impact system response, or how controller tuning parameters influence dynamic behavior. Rather than manually running each scenario one at a time, ANDES provides several approaches for batch processing that can dramatically reduce the time required for comprehensive studies.

This tutorial covers three approaches to batch simulation: file-based parallel processing for large studies, in-memory loops for smaller studies, and the `pool` option for intermediate cases where you want both parallelism and programmatic access to results.

:::{note}
**Prerequisites:** This tutorial assumes familiarity with Python fundamentals including loops, dictionaries, and NumPy arrays. Complete {doc}`02-first-simulation` through {doc}`07-eigenvalue-analysis` before proceeding.
:::

## Setup

In [ ]:
import andes
import numpy as np
from matplotlib import pyplot as plt
import os
import shutil

andes.config_logger(stream_level=30)  # Reduce logging verbosity for batch runs

## File-Based Batch Processing

For large parametric studies involving hundreds or thousands of scenarios, the most efficient approach is to generate separate case files and then run them in parallel using the ANDES command-line interface. This approach leverages multi-core processors effectively and allows you to checkpoint progress by checking which output files exist.

### Generating Case Files

The workflow begins by loading a base case, modifying parameters programmatically, and saving each variation to a new file. In this example, we create three cases with load levels varying from 100% to 120% of the base value.

In [ ]:
# Create output directory for the case files
os.makedirs('batch_cases', exist_ok=True)

# Load base case
kundur = andes.get_case('kundur/kundur_full.xlsx')
ss = andes.load(kundur)

In [ ]:
# Get the base load value
p0_base = ss.PQ.get('p0', 'PQ_0')
print(f"Base load: {p0_base:.2f} MW")

# Create cases with load varying from 100% to 120% of base
N_CASES = 3
p0_values = np.linspace(p0_base, 1.2 * p0_base, N_CASES)

for value in p0_values:
    ss.PQ.alter('p0', 'PQ_0', value)
    file_name = f'batch_cases/kundur_p_{value:.2f}.xlsx'
    andes.io.dump(ss, 'xlsx', file_name, overwrite=True)
    print(f"Created: {file_name}")

### Running Cases in Parallel

Once the case files are generated, you can run all of them in parallel using the `andes run` command with wildcards. ANDES automatically detects the number of CPU cores and distributes the workload across them.

In [ ]:
# Run all cases with time-domain simulation
!andes run batch_cases/*.xlsx -r tds

If you need to limit CPU usage (for example, to leave resources for other tasks on a shared workstation), use the `--ncpu` flag to specify the maximum number of parallel processes.

In [ ]:
# Limit to 2 parallel processes
!andes run batch_cases/*.xlsx -r tds --ncpu 2

### Returning System Objects for Post-Processing

When you want to analyze results programmatically after batch execution, use `pool=True` in the Python API. This runs cases in parallel and returns a list of System objects that you can then analyze or plot.

In [ ]:
# Run all cases and return System objects
systems = andes.run('batch_cases/*.xlsx', routine='tds', pool=True, verbose=30)

print(f"Completed {len(systems)} simulations")
print(f"Type of each result: {type(systems[0]).__name__}")

In [ ]:
# Plot results from all cases side by side
fig, axes = plt.subplots(1, len(systems), figsize=(12, 4))

for i, sys in enumerate(systems):
    sys.TDS.plt.plot(sys.GENROU.omega, ax=axes[i], 
                     title=f'Case {i+1}', latex=False, show=False)

plt.tight_layout()
plt.show()

## In-Memory Parameter Sweeps

For smaller studies where generating files would be unnecessary overhead, you can simply loop through parameter values in Python and accumulate results. This approach is particularly useful for quick exploratory analysis during model development or for sweeps involving only a few cases.

### Power Flow Parameter Sweep

The following example sweeps load levels and records the resulting bus voltage profiles. Since power flow converges quickly, this type of sweep can complete in seconds even for many parameter values.

In [ ]:
# Load system
ss = andes.run(kundur, no_output=True, default_config=True, verbose=30)

# View current load parameters
ss.PQ.as_df(vin=True)

In [ ]:
# Define parameter sweep range
n_samples = 5
pq0_values = np.linspace(10, 14, n_samples)  # MW range for PQ_0

# Storage for results: voltage at each bus for each parameter value
v_results = np.zeros((ss.Bus.n, n_samples))

# Run power flow for each load level
for i, p0 in enumerate(pq0_values):
    ss.PQ.alter('p0', 'PQ_0', p0)
    ss.PFlow.run()
    v_results[:, i] = ss.dae.y[ss.Bus.v.a]

print(f"Completed {n_samples} power flow calculations")

In [ ]:
# Visualize how voltage profiles change with load level
plt.figure(figsize=(10, 5))
for i in range(n_samples):
    plt.plot(v_results[:, i], 'o-', label=f'P0={pq0_values[i]:.1f} MW')

plt.xlabel('Bus Index')
plt.ylabel('Voltage [p.u.]')
plt.title('Voltage Profile vs Load Level')
plt.legend()
plt.xticks(range(ss.Bus.n), ss.Bus.name.v, rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

### Batch Time-Domain Simulation

The same looping approach works for time-domain simulations, though each iteration takes longer. This example demonstrates running multiple contingency scenarios with different line trips. For each scenario, we create a fresh System object, add the appropriate disturbance, run the simulation, and store the result.

In [ ]:
# Find available lines for contingency analysis
ss = andes.load(kundur, setup=False)
print(f"Available lines: {ss.Line.idx.v}")

# Select a subset for demonstration
lines_to_test = ss.Line.idx.v[:3]

In [ ]:
results = dict()

for line_idx in lines_to_test:
    # Load fresh system with setup=False to allow adding devices
    ss = andes.load(kundur, setup=False)
    
    # Add line trip at t=1.0s, restore at t=1.1s
    ss.add('Toggle', dict(model='Line', dev=line_idx, t=1.0))
    ss.add('Toggle', dict(model='Line', dev=line_idx, t=1.1))
    
    # Finalize system setup
    ss.setup()
    
    # Disable any existing Toggle from the test case
    ss.Toggle.alter('u', 1, 0.0)
    
    # Run simulation
    ss.PFlow.run()
    ss.TDS.config.tf = 5
    ss.TDS.config.no_tqdm = 1
    ss.TDS.run()
    
    results[line_idx] = ss
    print(f"Completed: {line_idx} trip")

In [ ]:
# Compare generator response across contingencies
fig, axes = plt.subplots(1, len(results), figsize=(12, 4))

for ax, (line_idx, ss) in zip(axes, results.items()):
    ss.TDS.plt.plot(ss.GENROU.omega, ax=ax, 
                    title=f'{line_idx} Trip', latex=False, show=False)

plt.tight_layout()
plt.show()

## Choosing the Right Approach

The following table summarizes when to use each batch processing technique:

| Approach | Best For | Parallelism | Memory Usage |
|----------|----------|-------------|---------------|
| File-based CLI | Large studies (>100 cases), production runs | Multi-process | Low (one case at a time) |
| `pool=True` API | Medium studies, need results in memory | Multi-process | High (all results in memory) |
| Python loop | Small studies, rapid prototyping | Single-thread | Controlled (can discard results) |

For studies with more than about 10 cases, the file-based parallel approach is usually fastest because it fully utilizes all CPU cores. For smaller studies or when you are actively developing and testing, the simpler Python loop avoids the overhead of file I/O.

## Cleanup

In [ ]:
shutil.rmtree('batch_cases', ignore_errors=True)
!andes misc -C

## Next Steps

- {doc}`09-contingency-analysis` - Systematic N-1 contingency screening
- {doc}`10-dynamic-control` - Runtime parameter modifications
- {doc}`11-frequency-response` - Frequency response analysis